Creating a SQL DB for customers in Sql lite

In [1]:
import pandas as pd
import sqlite3

In [10]:
#Libraries
from dotenv import load_dotenv
import os

#Loading Environment Vars
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "ConversationalAIBot"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

In [11]:
df = pd.read_csv("telecom_customer_table.csv", encoding="ISO-8859-1")

df.head(3)

,Customer ID,Customer Name,Age,Gender,Location,Plan Subscribed,Device Used,Plan Details,Network Type,Join Date,Recent Issue Reported,Response Provided
0,CUST1000,Ricky Ward,44,Non-Binary,Sethhaven,Basic Plan,Samsung Galaxy S22,"Costs $20to$30/month, includes 5 GB high-spee...",LTE,21-02-2025,Slow internet speed,We are sorry. We will ship a new one.
1,CUST1001,Mallory Graham,18,Female,West Garymouth,Unlimited Plan,Samsung Galaxy S22,"Costs $30to$50/month, truly unlimited data (F...",LTE,26-09-2024,Forgot password,We're expanding network towers to improve cove...
2,CUST1002,Travis Howe,25,Male,Kimberlyton,Family Plan,iPhone 14,"Costs $60to$100/month (3to5 lines), shared 5...",LTE,13-02-2024,Cannot make international calls,We have escalated the voicemail issue to our t...


In [12]:
#Sample 100 rows is used here and pretty much the columns are self expalanatory
connection = sqlite3.connect("telecom_customer.db")
df.to_sql("customers", connection, if_exists="replace", index=False)

100

In [13]:
connection.close()

In [14]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///telecom_customer.db")

Using Langchain SQL agent

In [16]:
from langchain_groq import ChatGroq
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import initialize_agent, AgentType
from langchain_core.prompts import PromptTemplate

model = ChatGroq(
    temperature = 0.2,
    model_name = "llama-3.3-70b-versatile",
)

In [17]:
SQL_toolkit = SQLDatabaseToolkit(db=db, llm=model)

In [18]:
sql_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=SQL_toolkit.get_tools(),
    llm=model,
    verbose=True
)

C:\Users\bharath.sr.lv\AppData\Local\Temp\ipykernel_35340\3930081775.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  sql_agent = initialize_agent(


In [19]:
customer_id = "CUST1001"  
query = f"What are the details of customer with Customer ID {customer_id}?"

response = sql_agent.run(query)
print(response)

C:\Users\bharath.sr.lv\AppData\Local\Temp\ipykernel_35340\2590517207.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = sql_agent.run(query)




> Entering new AgentExecutor chain...
To find the details of a customer with a specific Customer ID, I first need to know the structure of the tables in the database, particularly the ones related to customers. 

Action: sql_db_list_tables
Action Input: 
Observation: customers
Thought:Since I now know that there is a table named "customers", I can use the sql_db_schema tool to get the schema and sample rows of this table.

Action: sql_db_schema
Action Input: customers
Observation: 
CREATE TABLE customers (
	"Customer ID" TEXT, 
	"Customer Name" TEXT, 
	"Age" INTEGER, 
	"Gender" TEXT, 
	"Location" TEXT, 
	"Plan Subscribed" TEXT, 
	"Device Used" TEXT, 
	"Plan Details" TEXT, 
	"Network Type" TEXT, 
	"Join Date" TEXT, 
	"Recent Issue Reported" TEXT, 
	"Response Provided" TEXT
)

/*
3 rows from customers table:
Customer ID	Customer Name	Age	Gender	Location	Plan Subscribed	Device Used	Plan Details	Network Type	Join Date	Recent Issue Reported	Response Provided
CUST1000	Ricky Ward	44	Non-Bin